# Capstone project

The goal of this project is to find areas with open venues around a given location at given time of the day

Input:

- US address
- Day of the week
- Time of the day
- Radius

Output:

A map of venues opened at that day and time

Logic:

- User OpenCage service API to map te address to lan/lat coordinates
- Find venues in the area using Foursquare API
- For every venue download its schedule. It's a premium call, so the free account will allow only a limited number
- Check if that venue is going to be open during the requested day/time and set a flag
- Clean up data and leave only the open venues
- Map using Folium


## Input data

Target location. Enter a US address.
Target day. Day of the week 1-7
Target time. "0000"-"2359"
Target radius. In meters

In [413]:
target={}
target['address']="6200 Stoneridge Mall Rd, Pleasanton, CA"
target['name']="Office in Pleasanton"
target['day']=3
target['time']="0030"
# These values are optional
target['radius']=1000 # Keep it walkable, about 1 mile
target['latitude']=None # We'll find out later based on the address
target['longtitude']=None # We'll find out later based on the address


## Imports

In [414]:
import numpy as np
import pandas as pd
import folium 
import requests
from opencage.geocoder import OpenCageGeocode

## Definitions

In [428]:
GEOCODERKEY = 'cce05175e31c445c8f4b02445d1215b6' # Geocoder Key

CLIENT_ID = '415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW' # your Foursquare ID
CLIENT_SECRET = 'A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G' # your Foursquare Secret
VERSION = '20200125' # Foursquare API version

LIMIT=100 # Max numer of returned venues

CATEGORIES=[
    '4d4b7104d754a06370d81259', # Arts & Entertainment
    '4d4b7105d754a06373d81259', # Event
    '4d4b7105d754a06374d81259', # Food
    '4d4b7105d754a06376d81259', # Nightlife spot
]

print('Your credentails:')
print('GEOCODERKEY: ' + GEOCODERKEY)
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
GEOCODERKEY: cce05175e31c445c8f4b02445d1215b6
CLIENT_ID: 415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW
CLIENT_SECRET:A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G


## Libraries

In [417]:
def getNearbyVenuesCategoriesLocation(categories, latitude, longitude, radius=500):
    
    venues_list=[]
    for categoryId in categories:
        #print(categoryId)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            categoryId)
        #print(url)   
        # make the GET request
        r = requests.get(url).json()
        #print("r:",r)
        if r["response"]['groups']:
            results=r["response"]['groups'][0]['items']
        else:
            return []
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            categoryId,
            v['venue']['id'],
            False, # Is it open at the target time? just a placeholder column for now, we'll find out later
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['CategoryId', 
                  'VenueId', 
                  'Open', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Main program

In [ ]:
First, we need to convert address into lat/lon. We are going to use free locationIQ.com API

In [418]:
geocoder = OpenCageGeocode(GEOCODERKEY)

results = geocoder.geocode(target['address'])

print(u'%f;%f;%s;%s' % (results[0]['geometry']['lat'], 
                        results[0]['geometry']['lng'],
                        results[0]['components']['country_code'],
                        results[0]['annotations']['timezone']['name']))

target['latitude']=results[0]['geometry']['lat'] 
target['longtitude']=results[0]['geometry']['lng'] 

print(target)

37.695754;-121.924314;us;America/Los_Angeles
{'address': '6200 Stoneridge Mall Rd, Pleasanton, CA', 'name': 'Office in Pleasanton', 'day': 3, 'time': '0030', 'radius': 1000, 'latitude': 37.6957537, 'longtitude': -121.9243137}


Let's pull all venues in the area. For now we assume the are all closed.

In [419]:
target_venues = getNearbyVenuesCategoriesLocation(
                                categories=CATEGORIES,   
                                latitude=target['latitude'],
                                longitude=target['longtitude'],
                                radius=target['radius']
                                  )

In [420]:
target_venues.head(10)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4d4b7104d754a06370d81259,5832f5aa44587f7bebacfb3a,False,Click Art Museum,37.696267,-121.928818,Art Gallery
1,4d4b7104d754a06370d81259,4ae3cf7bf964a520189921e3,False,Bunjo's Comedy Lounge,37.700374,-121.931879,Comedy Club
2,4d4b7104d754a06370d81259,53f7efa4498e573ad1533935,False,Kinderdance of the Tri-Valley,37.703276,-121.930302,Dance Studio
3,4d4b7105d754a06374d81259,51e05298498ebd2166dccccf,False,Food Trucks @ Stoneridge Mall,37.694091,-121.925186,Food Truck
4,4d4b7105d754a06374d81259,4b648623f964a520aaba2ae3,False,Andersen Bakery,37.695850,-121.929117,Bakery
5,4d4b7105d754a06374d81259,50bebf13e4b0520fb9a76416,False,La Boulangerie,37.695765,-121.928601,Bakery
6,4d4b7105d754a06374d81259,48277877f964a520b44f1fe3,False,P.F. Chang's,37.693527,-121.928858,Chinese Restaurant
7,4d4b7105d754a06374d81259,4b4143daf964a52057c425e3,False,Auntie Anne's,37.695061,-121.928375,Snack Place
8,4d4b7105d754a06374d81259,4e59a2f7d22dfba6f477bac8,False,California Pizza Kitchen,37.694440,-121.929077,Pizza Place
9,4d4b7105d754a06374d81259,4ccdec87ba79a1cdc53744cb,False,Espresso Bar,37.696246,-121.927938,Diner


In [421]:
target_venues.groupby('Venue Category').count()

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,2,2,2,2,2,2
Art Gallery,1,1,1,1,1,1
BBQ Joint,1,1,1,1,1,1
Bakery,3,3,3,3,3,3
Bowling Alley,1,1,1,1,1,1
Burger Joint,1,1,1,1,1,1
Burrito Place,1,1,1,1,1,1
Café,5,5,5,5,5,5
Chinese Restaurant,2,2,2,2,2,2


In [422]:
target_venues['VenueId'].count()

48

Ok, now we need to pull hours for every venue and set the open Flag based on the schedule

Unfortunately, the response is a bit fuzzy. The documentation is available at https://developer.foursquare.com/docs/api/venues/hours .

There are two kinds of schedules:

hours: as I understood, this is when the venue is supposed to be open.
popular: this is when people actually go there.

We can get both values returned, one of them or none. So we need to define priorities. For the purpose of this project, the logic is going to be:

if target day/time in 'popular' OR 'hours' - mark venue Open=True

That means if there are conflicting data - we select Open.
If both 'popular' and 'hours' are not present - that would keep the value of Open set to False (the default).

A function to check if a given day/time falls into open window of a response:

In [423]:

def isOpen(req,day,time):
    """
    Checking the schedule in the request req against the day and the time
    return True if either hours or popular include that day and time, otherwise False
    Foursquare uses the "+0100" to indicate the venue stays open after midnight
    """
    #print(day,time,req)
    
    curr_day = day
    prev_day = day-1 if day>1 else 7
    
    # Checking the previous day:
    if tr_1['response']['popular']:
        for i in tr_1['response']['popular']['timeframes']:
            #print(i['days'])
            if prev_day in i['days']:
                #print(i)
                for o in i['open']:
                    #print("open: ",o)
                    if o['end'].startswith('+'):
                        if int(time)<int(o['end'][1:]):
                            #print("Match!")
                            return True
    # Checking the current day:
    if tr_1['response']['popular']:
        for i in tr_1['response']['popular']['timeframes']:
            #print(i['days'])
            if curr_day in i['days']:
                #print(i)
                for o in i['open']:
                    #print("open: ",o)
                    if int(o['start'])<=int(time):
                        if int(time)<int( '2359' if o['end'].startswith('+') else o['end']):
                            #print("Match!")
                            return True
    
    # Checking the previous day:
    if tr_1['response']['hours']:
        for i in tr_1['response']['hours']['timeframes']:
            #print(i['days'])
            if prev_day in i['days']:
                #print(i)
                for o in i['open']:
                    #print("open: ",o)
                    if o['end'].startswith('+'):
                        if int(time)<int(o['end'][1:]):
                            #print("Match!")
                            return True
                        
    # Checking the current day:
    if tr_1['response']['hours']:
        for i in tr_1['response']['hours']['timeframes']:
            #print(i['days'])
            if curr_day in i['days']:
                #print(i)
                for o in i['open']:
                    #print("open: ",o)
                    if int(o['start'])<=int(time):
                        if int(time)<int( '2359' if o['end'].startswith('+') else o['end']):
                            #print("Match!")
                            return True
    return False
    
#isOpen(tr_1,target['day'],target['time'])
#isOpen(tr_1,t_day,t_time)

Tests of isOpen:

In [296]:
tr_1= {'meta': 
       {'code': 200, 'requestId': '5e2cddb1c546f3001b7f5085'}, 
       'response': {
           'hours': {}, 
           'popular': {'timeframes': [
               {'days': [6], 'includesToday': True, 'open': [{'start': '1400', 'end': '1500'}, {'start': '1900', 'end': '+0200'}], 'segments': []}, 
               {'days': [7], 'open': [{'start': '1400', 'end': '1800'}, {'start': '2100', 'end': '+0000'}], 'segments': []}, 
               {'days': [1], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1700', 'end': '+0200'}], 'segments': []}, 
               {'days': [2], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1800', 'end': '+0200'}], 'segments': []}, 
               {'days': [3], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1700', 'end': '+0100'}], 'segments': []}, 
               {'days': [4], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1200', 'end': '1300'}, {'start': '1700', 'end': '+0100'}], 'segments': []}, 
               {'days': [5], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1700', 'end': '+0200'}], 'segments': []}]}}}
t_day=3
t_time='0600'

#isOpen(tr_1,target['day'],target['time'])
#isOpen(tr_1,t_day,t_time)

False

In [424]:
def setOpenFlagForVenues(venues,day,time):
    for venue_id in venues['VenueId']:
        if venue_id:
            print(venue_id)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/{}/hours?&client_id={}&client_secret={}&v={}'.format(
                venue_id,
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION)
            print(url)   
            # make the GET request
            r = requests.get(url).json()
            print("r:",r)
            #print("hours: ",r['response']['hours'])
            #print("popular: ",r['response']['popular'])
            #print(venue_id)
            #print(venues['VenueID':venue_id])
            venues.loc['Open','venue_id']=isOpen(r,day,time)
    #r=tr_1
    #venue_id='5402a13f498e6650fc99ef2d'
    #print(r)
    #print(isOpen(r,day,time))
    #venues.loc[venues['VenueId'] == venue_id, ['Open']] = True #isOpen(r,day,time)
    #print(venues)

    return

In [425]:
target_venues.head(15)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4d4b7104d754a06370d81259,5832f5aa44587f7bebacfb3a,False,Click Art Museum,37.696267,-121.928818,Art Gallery
1,4d4b7104d754a06370d81259,4ae3cf7bf964a520189921e3,False,Bunjo's Comedy Lounge,37.700374,-121.931879,Comedy Club
2,4d4b7104d754a06370d81259,53f7efa4498e573ad1533935,False,Kinderdance of the Tri-Valley,37.703276,-121.930302,Dance Studio
3,4d4b7105d754a06374d81259,51e05298498ebd2166dccccf,False,Food Trucks @ Stoneridge Mall,37.694091,-121.925186,Food Truck
4,4d4b7105d754a06374d81259,4b648623f964a520aaba2ae3,False,Andersen Bakery,37.695850,-121.929117,Bakery
5,4d4b7105d754a06374d81259,50bebf13e4b0520fb9a76416,False,La Boulangerie,37.695765,-121.928601,Bakery
6,4d4b7105d754a06374d81259,48277877f964a520b44f1fe3,False,P.F. Chang's,37.693527,-121.928858,Chinese Restaurant
7,4d4b7105d754a06374d81259,4b4143daf964a52057c425e3,False,Auntie Anne's,37.695061,-121.928375,Snack Place
8,4d4b7105d754a06374d81259,4e59a2f7d22dfba6f477bac8,False,California Pizza Kitchen,37.694440,-121.929077,Pizza Place
9,4d4b7105d754a06374d81259,4ccdec87ba79a1cdc53744cb,False,Espresso Bar,37.696246,-121.927938,Diner


In [427]:
setOpenFlagForVenues(target_venues,target['day'],target['time'])

5832f5aa44587f7bebacfb3a
https://api.foursquare.com/v2/venues/5832f5aa44587f7bebacfb3a/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
r: {'meta': {'code': 200, 'requestId': '5e2e7940760a7f001b7e06ec'}, 'response': {'hours': {'timeframes': [{'days': [1, 2, 3, 4, 5, 6], 'open': [{'start': '1000', 'end': '2100'}], 'segments': []}, {'days': [7], 'includesToday': True, 'open': [{'start': '1100', 'end': '1900'}], 'segments': []}]}, 'popular': {}}}
4ae3cf7bf964a520189921e3
https://api.foursquare.com/v2/venues/4ae3cf7bf964a520189921e3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e792db4b684001b7131a1'}, 'response': {}}
53f7efa4498e573ad1533935
https://api.foursquare.com/v2/venues/53f7efa4498e573ad1533935

r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e7992edbcad001e0a733b'}, 'response': {}}
52316975bce6b2bf812afeaa
https://api.foursquare.com/v2/venues/52316975bce6b2bf812afeaa/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e79bc47b43d0dca924829'}, 'response': {}}
50b41d84e4b0818e9a8d2353
https://api.foursquare.com/v2/venues/50b41d84e4b0818e9a8d2353/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e798a83525f001b03f311'}, 'response': {}}
4bff16b3ca1920a1f35bec81
https://api.foursquare.com/v2/venues/4bff16b3ca1920a1f35bec81/hours?&client_i

r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e79b0949393001ba3ba22'}, 'response': {}}
4afa2ca3f964a5202b1722e3
https://api.foursquare.com/v2/venues/4afa2ca3f964a5202b1722e3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e79a5216785001b956780'}, 'response': {}}
56343ca3498e6c1e11e17ba6
https://api.foursquare.com/v2/venues/56343ca3498e6c1e11e17ba6/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
r: {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2e786502a172002611f186'}, 'response': {}}
53965a85498eb27aa27ca6cc
https://api.foursquare.com/v2/venues/53965a85498eb27aa27ca6cc/hours?&client_i

In [401]:
target_venues.head(10)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category,venue_id
0,4d4b7104d754a06370d81259,596ad8168173cb7da55c3925,False,K1 Speed,37.705923,-121.916341,Go Kart Track,NaN
1,4d4b7104d754a06370d81259,5832f5aa44587f7bebacfb3a,False,Click Art Museum,37.696267,-121.928818,Art Gallery,NaN
2,4d4b7104d754a06370d81259,4ae3cf7bf964a520189921e3,False,Bunjo's Comedy Lounge,37.700374,-121.931879,Comedy Club,NaN
3,4d4b7104d754a06370d81259,53f7efa4498e573ad1533935,False,Kinderdance of the Tri-Valley,37.703276,-121.930302,Dance Studio,NaN
4,4d4b7104d754a06370d81259,4df152aa52b100c2d7f150f9,False,Pleasanton Outdoor Roller Hockey Rink,37.689140,-121.914061,Roller Rink,NaN
5,4d4b7104d754a06370d81259,4ccf472df6378cfa9499add6,False,Dublin Heritage Park & Museum,37.700634,-121.938182,History Museum,NaN
6,4d4b7104d754a06370d81259,57ecba99cd10d7db9fe3147e,False,4Ever Dance Studio,37.704979,-121.935186,Dance Studio,NaN
7,4d4b7105d754a06374d81259,51e05298498ebd2166dccccf,False,Food Trucks @ Stoneridge Mall,37.694091,-121.925186,Food Truck,NaN
8,4d4b7105d754a06374d81259,4b648623f964a520aaba2ae3,False,Andersen Bakery,37.695850,-121.929117,Bakery,NaN
9,4d4b7105d754a06374d81259,50bebf13e4b0520fb9a76416,False,La Boulangerie,37.695765,-121.928601,Bakery,NaN


Ok, at this point target_venues have flag 'Open' set to False or True. We want to leave only the Open ones

In [402]:
target_venues.drop( target_venues[ target_venues['Open'] == False ].index , inplace=True)

In [403]:
target_venues.head(10)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category,venue_id
Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


At this point we know all open venues.

Let's show them on the map

In [383]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map of New York using latitude and longitude values
map_target = folium.Map(location=[target['latitude'], target['longtitude']], zoom_start=15)

colors_array = cm.rainbow(np.linspace(0, 1, len(CATEGORIES)+3))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, name, categoryId, category in zip(target_venues['Venue Latitude'], target_venues['Venue Longitude'], target_venues['Venue'], target_venues['CategoryId'], target_venues['Venue Category']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color=rainbow[CATEGORIES.index(categoryId)],
        fill=True,
        fill_color=rainbow[CATEGORIES.index(categoryId)],
        fill_opacity=0.7,
        parse_html=False).add_to(map_target)  

# add the target

label = '{}'.format(target['name'])
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [target['latitude'], target['longtitude']],
    radius=8,
    popup=label,
    color=rainbow[-1],
    fill=True,
    fill_color=rainbow[-1],
    fill_opacity=0.7,
    parse_html=False).add_to(map_target) 
    
map_target